# Fish-invertebrate associations at Cypress Point, Monterey Peninsula, CA

### Data

The data are in two .csv files exported from the Microsoft Access database for the MARE ROV surveys from 2005–2019. `Fish_1sec_2005-2019_2-3-2021_CP.csv` contains fish species data and `Invert_1sec_2005-2019_2-3-2021_CP.csv` contains invertebrate taxonomic data.

In [34]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt

In [67]:
#load data
fish = pd.read_csv("Fish_1sec_205-2019_2-3-2021_CP.csv")
inverts = pd.read_csv("Invert_1sec_2005-2019_2-3-2021_CP.csv")

In [68]:
#Remove unusable data and remove 2009 survey from fish data (because there is no corresponding 2009 invertebrate data)
fish = fish[fish['Unusable_Data']!='X']
fish = fish[fish['Survey_Year']!='2009']
inverts = inverts[inverts['Unusable_Data']!='X']

Next we will create new columns based on habitat type. We shift the habitat type column by one row, then use boolean logic to differentiate where new habitat patches (continuous sets of data within the same habitat type) occur. Values in the resulting `Habitat_patch` columns can be interpreted as the habitat patch identifiers. See [this resource](https://towardsdatascience.com/pandas-dataframe-group-by-consecutive-same-values-128913875dba).

In [69]:
fish['Habitat_shift'] = fish['Habitat_Type'].shift()
fish['shift!=orig'] = np.where(fish['Habitat_shift']!=fish['Habitat_Type'],True,False)
fish['Habitat_patch'] = np.cumsum(fish['shift!=orig'])

inverts['Habitat_shift'] = inverts['Habitat_Type'].shift()
inverts['shift!=orig'] = np.where(inverts['Habitat_shift']!=inverts['Habitat_Type'],True,False)
inverts['Habitat_patch'] = np.cumsum(inverts['shift!=orig'])

Next we will assign an identifier to each row based on the transect that it is in. This is so that we can count how many distinct habitat patches exist in each transect for both fish and invert data, and insert zeroes as necessary to make the number of habitat patches constant between both.

In [78]:
fish['Transect_shift'] = fish["Transect_ID"].shift()
fish['transectshift!=orig'] = np.where(fish['Transect_shift']!=fish["Transect_ID"],True,False)
fish['Transect_no'] = np.cumsum(fish['transectshift!=orig'])

inverts['Transect_shift'] = inverts["Transect_ID"].shift()
inverts['transectshift!=orig'] = np.where(inverts['Transect_shift']!=inverts["Transect_ID"],True,False)
inverts['Transect_no'] = np.cumsum(inverts['transectshift!=orig'])
fish

,Project,Region,Survey_Year,Survey_Date,Location,MPA_Group,Type,Designation,Site,Dive,Line,Transect_ID,Habitat_Type,Lat,Lon,Depth,Cumulative Dist (m),Cumulative Area (m),TC,Sec,Taxserial,CommonName,SpeciesName,Count,Sex,Unusable_Data,Habitat_shift,shift!=orig,Habitat_patch,Transect_shift,transectshift!=orig,Transect_no
0,CCMPA 2008,Central,2008,10/23/2008,Cypress Point,Point Lobos,Reference,Reference,PL11,430,1300,CCMPA 2008_430_1300,Soft,36.568232,-121.992699,101.6,2.22,4.53,15:25:59,278090759,166762,Halfbanded Rockfish,Sebastes semicinctus,1.0,U,NaN,NaN,True,1,NaN,True,1
1,CCMPA 2008,Central,2008,10/23/2008,Cypress Point,Point Lobos,Reference,Reference,PL11,430,1300,CCMPA 2008_430_1300,Soft,36.568250,-121.992707,100.7,4.42,9.68,15:26:08,278090768,172859,UI flatfish,Unidentified Pleuronectidae,1.0,U,NaN,Soft,False,1,CCMPA 2008_430_1300,False,1
2,CCMPA 2008,Central,2008,10/23/2008,Cypress Point,Point Lobos,Reference,Reference,PL11,430,1300,CCMPA 2008_430_1300,Soft,36.568296,-121.992811,100.2,16.51,42.23,15:26:40,278090800,172859,UI flatfish,Unidentified Pleuronectidae,1.0,U,NaN,Soft,False,1,CCMPA 2008_430_1300,False,1
3,CCMPA 2008,Central,2008,10/23/2008,Cypress Point,Point Lobos,Reference,Reference,PL11,430,1300,CCMPA 2008_430_1300,Soft,36.568334,-121.992801,100.3,21.19,53.73,15:26:51,278090811,172859,UI flatfish,Unidentified Pleuronectidae,1.0,U,NaN,Soft,False,1,CCMPA 2008_430_1300,False,1
4,CCMPA 2008,Central,2008,10/23/2008,Cypress Point,Point Lobos,Reference,Reference,PL11,430,1300,CCMPA 2008_430_1300,Soft,36.568593,-121.992617,97.7,56.99,185.48,15:27:48,278090868,172859,UI flatfish,Unidentified Pleuronectidae,1.0,U,NaN,Soft,False,1,CCMPA 2008_430_1300,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7551,CAMPA 2019,Central,2019,08/14/2019,Cypress Point,Point Lobos,Reference,Reference,PL11,34,1010,CAMPA 2019_34_1010,Hard,36.572738,-121.991048,74.7,371.06,823.89,17:15:49,619118149,999996,YOY,Young of year (<10 cm Sebastes sp.),4.0,U,NaN,Hard,False,717,CAMPA 2019_34_1010,False,43
7552,CAMPA 2019,Central,2019,08/14/2019,Cypress Point,Point Lobos,Reference,Reference,PL11,34,1010,CAMPA 2019_34_1010,Hard,36.572740,-121.991048,74.8,371.32,824.32,17:15:50,619118150,999996,YOY,Young of year (<10 cm Sebastes sp.),4.0,U,NaN,Hard,False,717,CAMPA 2019_34_1010,False,43
7553,CAMPA 2019,Central,2019,08/14/2019,Cypress Point,Point Lobos,Reference,Reference,PL11,34,1010,CAMPA 2019_34_1010,Hard,36.572742,-121.991049,74.8,371.57,824.67,17:15:51,619118151,999996,YOY,Young of year (<10 cm Sebastes sp.),4.0,U,NaN,Hard,False,717,CAMPA 2019_34_1010,False,43
7554,CAMPA 2019,Central,2019,08/14/2019,Cypress Point,Point Lobos,Reference,Reference,PL11,34,1010,CAMPA 2019_34_1010,Hard,36.572742,-121.991049,74.8,371.57,824.67,17:15:51,619118151,166738,Rosy Rockfish,Sebastes rosaceus,1.0,U,NaN,Hard,False,717,CAMPA 2019_34_1010,False,43


Note: I would like to do the analysis based on habitat patches, but splitting the data so that patches match between invert and fish datasets is very complicated, so for the time being I will analyze using transects as replicates. I would like to ask Tom for help on the habitat patch data splitting.

In [92]:
fish.groupby(['Transect_no', 'Habitat_Type', 'SpeciesName']).agg({'Count':sum})

Count
Transect_no Habitat_Type SpeciesName                                
1           Hard         Sebastes hopkinsi                     31.00
                         Sebastes miniatus                      1.00
                         Sebastes mystinus                      6.00
                         Sebastes serranoides or flavidus       2.00
                         Unidentified Sebastes sp.            142.00
...                                                              ...
43          Soft         Young of year (<10 cm Sebastes sp.)   87.34
            Soft_Mixed   Sebastes constellatus                  1.00
                         Sebastes rosaceus                      8.00
                         Sebastes semicinctus                  13.00
                         Young of year (<10 cm Sebastes sp.)    6.67

[773 rows x 1 columns]

In [87]:
#fish.groupby(['Transect_no']).agg({'Habitat_patch':'unique'})
#for i in fish['Transect_no']:
    #print(i)

In [86]:
#inverts.groupby(['Transect_no']).agg({'Habitat_patch':'unique'})

In [ ]:
#Sum species counts within each habitat patch
#fishpatches = fish.groupby(['Habitat_patch', 'Habitat_Type','TC', 'SpeciesName']).agg({'Count':sum})
#invertpatches = inverts.groupby(['Habitat_patch', 'Habitat_Type','TC', 'SpeciesName']).agg({'Count':sum})
#invertpatches

In [ ]:
#Total abundance within each habitat patch
#fishpatchesall = fish.groupby(['Habitat_patch', 'Habitat_Type']).agg({'Count':sum})
#invertpatchesall = inverts.groupby(['Habitat_patch', 'Habitat_Type']).agg({'Count':sum})

In [ ]:
#split data up into separate dataframes for each habitat type (hard, hard mixed, soft mixed, soft)
inv_hard = inverts[inverts['Habitat_Type']=="Hard"]
inv_hardmix = inverts[inverts['Habitat_Type']=="Hard_Mixed"]
inv_softmix = inverts[inverts['Habitat_Type']=="Soft_Mixed"]
inv_soft = inverts[inverts['Habitat_Type']=="Soft"]

fish_hard = fish[fish['Habitat_Type']=="Hard"]
fish_hardmix = fish[fish['Habitat_Type']=="Hard_Mixed"]
fish_softmix = fish[fish['Habitat_Type']=="Soft_Mixed"]
fish_soft = fish[fish['Habitat_Type']=="Soft"]

In [ ]:
for row in fish_hard:
    if fish_hard['Habitat_patch']>inv_hard['Habitat_patch']:
        newrow = DataFrame({'Habitat_patch': 30.0, "length": 1.3}, index=[3])
        df2 = concat([df.iloc[:2], line, df.iloc[2:]]).reset_index(drop=True)

In [ ]:
#sum species counts
#inv_hard = inv_hard.groupby(['SpeciesName']).sum()
#inv_hardmix = inv_hardmix.groupby(['SpeciesName']).sum()
#inv_softmix = inv_softmix.groupby(['SpeciesName']).sum()
#inv_soft = inv_soft.groupby(['SpeciesName']).sum()

#fish_hard = fish_hard.groupby(['SpeciesName']).sum()
#fish_hardmix = fish_hardmix.groupby(['SpeciesName']).sum()
#fish_softmix = fish_softmix.groupby(['SpeciesName']).sum()
#fish_soft = fish_soft.groupby(['SpeciesName']).sum()

In [ ]:
inverts = inverts.groupby((inverts['Habitat_Type'].shift() != inverts['Habitat_Type']).cumsum())
inverts

In [ ]:
plt.figure()
plt.plot(sum(inv_hard['Count']), sum(fish_hard['Count']), '.')